# Homework 3: More String, Lists, and Python Objects (18 pts)
**STATS 507, Fall 2025**

-------

**Name: YI WANG** 

**Email: wangyii@umich.edu** 

**Time spent on this homework: 3 hours** 

**I did not discuss this homework with anyone.**


In [1]:
import collections
import string
import itertools

---
## Question 1: Infinite sequences (8 pts)
For each of the problems below, write a function which generates the given infinite sequence. We should be able to use your generators to access any entry of the sequence no matter how deep.

**1(a)** (2 pts) The prime numbers are

$$2, 3, 5, 7, 11, 13, 17, \dots$$

Give a generator for the primes. (Note: many algorithms exist for this problem. Yours should be efficient enough that we can use it to generate reasonably large prime numbers.)

In [2]:
def primes():
    yield 2

    primes_found = [2]
    candidate = 3
    
    while True:
        is_prime = True

        for p in primes_found:
            if p * p > candidate:
                break
            if candidate % p == 0:
                is_prime = False
                break
        
        if is_prime:
            primes_found.append(candidate)
            yield candidate
        
        candidate += 2  

In [3]:
"""Check that squares returns the correct output for several inputs"""
p = primes()
plist = [2,3,5,7,11,13,17,19,23,29]
for i in range(10):
    assert next(p) == plist[i]

**1(b)** (3 pts) The *ruler sequence* is

$$1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7\dots$$

The first number (1) appears once; the next two numbers (2 and 3) appear twice, the next three numbers appear three times, etc.

Hint: You are highly encouraged to use relevant functions from itertools to make your solution simple

In [4]:
def ruler():
    group_size = 1
    num = 1
    
    while True:
        for _ in range(group_size):
            yield from itertools.repeat(num, group_size)
            num += 1
        group_size += 1

In [5]:
r = ruler()
ruler_numbers = [1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8]
for n in ruler_numbers:
    assert next(r) == n

**1(c)** (3 pts) The look-and-say sequence  

$$1, 11, 21, 1211, 111221, 312211, 13112221, 1113213211, \dots$$

is generated as follows:

1. The first entry is one.
2. The second entry is generated by reading the first entry aloud: "one one"
3. The third entry is generated by reading the second entry aloud: "two ones"
4. The fourth entry is generated by reading the third entry aloud: "one two and one one"
5. The fifth entry is therefore "one one, one two, and two ones"

... and so forth. Note that each entry of the sequence should be a `str` object.

In [6]:
def look_say():
    current = "1"
    
    while True:
        yield current

        next_term = ""
        i = 0
        
        while i < len(current):
            digit = current[i]
            count = 1
            
            while i + count < len(current) and current[i + count] == digit:
                count += 1

            next_term += str(count) + digit
            i += count
        
        current = next_term

In [7]:
s = look_say()
look_say_first_ten = ['1',
 '11',
 '21',
 '1211',
 '111221',
 '312211',
 '13112221',
 '1113213211',
 '31131211131221',
 '13211311123113112211']
for i in range(10):
    s_i = next(s)
    assert isinstance(s_i, str)
    assert s_i == look_say_first_ten[i]

### Question 2: Simple ciphers (5 pts)

A *cipher* is an algorithm for encrypting or decrypting a text message, called *plaintext*, into *ciphertext*. In this exercise, we will assume that all plaintext to be encrypted or decrypted are composed of the lower-case Roman alphabet, without any punctuation or whitespace. Examples of such messages could be

```attackatdawn```

or 

```iamajellyfilleddonut```.

In [8]:
def enc_caesar(s: str, k: int) -> str:
    '''
    Encrypt the message s using Ceasar cipher with key k.
    
    >>> enc_caeasar('thequickbrownfoxjumpsoverthelazydog', 1)
    'uifrvjdlcspxogpykvnqtpwfsuifmbazeph'
    '''
    result = ""
    
    for char in s:
        char_num = ord(char) - ord('a')
        
        shifted_num = (char_num + k) % 26

        shifted_char = chr(shifted_num + ord('a'))
        result += shifted_char
    
    return result

In [9]:
public_tests = [('thequickbrownfoxjumpsoverthelazydog', 1, 'uifrvjdlcspxogpykvnqtpwfsuifmbazeph'), ('howdoesthiscodehandlewrapping', 18, 'zgovgwklzakugvwzsfvdwojshhafy'),('thekeycanbenegative', -3, 'qebhbvzxkybkbdxqfsb')]
for msg, k, enc in public_tests:
    assert enc_caesar(msg, k) == enc

In [10]:
def dec_caesar(s: str, k: int) -> str:
    '''
    Decrypt the message s using Ceasar cypher with key k.
    
    >>> dec_caeasar('uifrvjdlcspxogpykvnqtpwfsuifmbazeph', 1)
    'thequickbrownfoxjumpsoverthelazydog'
    '''
    result = ""
    
    for char in s:
        char_num = ord(char) - ord('a')
        
        shifted_num = (char_num - k) % 26
        
        shifted_char = chr(shifted_num + ord('a'))
        result += shifted_char
    
    return result

In [11]:
public_tests = [('thequickbrownfoxjumpsoverthelazydog', 1, 'uifrvjdlcspxogpykvnqtpwfsuifmbazeph'), ('howdoesthiscodehandlewrapping', 18, 'zgovgwklzakugvwzsfvdwojshhafy'),('thekeycanbenegative', -3, 'qebhbvzxkybkbdxqfsb')]
for msg, k, enc in public_tests: 
    assert dec_caesar(enc, k) == msg

### Question 3 (5 pts) Valid String.
Given a string s containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

An input string is valid if:

1) Open brackets must be closed by the same type of brackets.
2) Open brackets must be closed in the correct order.
3) Every close bracket has a corresponding open bracket of the same type.


In [12]:
def is_valid(s):
    stack = []

    bracket_map = {')': '(', '}': '{', ']': '['}
    
    for char in s:
        if char in bracket_map: 
        
            if not stack or stack.pop() != bracket_map[char]:
                return False
        else:  
            stack.append(char)
    
    return len(stack) == 0

In [13]:
assert is_valid('()') == True
assert is_valid('([{]}') == False
assert is_valid('([{}])') == True